In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import requests
import csv

In [2]:
#GETTING SLUGS FOR ALL THE PRODUCT PAGES

#am able to just do this with requests and not selenium because they have a small amount of products
#if it increases or pages are added this code will need to be altered

#categories the products fall under on their page
categories = ['cleansers',
             'toners',
             'moisturizers',
             'treatments-and-serums',
             'masks-exfoliants',
             'eye-care',
             'lip-care',
             'sun-care']

slugs_df = pd.DataFrame(columns = ['category', 'href'])   
for category in categories:
    time.sleep(20)
    url = "https://follain.com/collections/skincare-" + category + '?sort_by=best-selling'
    res = requests.get(url)
    
    #if there is a fail attempt print the url that's not working and break
    if res.status_code != 200:
        print('error when requestion {}'.format(url))
        break
    soup = BeautifulSoup(res.content, 'html.parser')
    page = soup.find_all('div', {'class':"product-grid-item-inner"})
    slugs = []
    #pulling the individual slugs for each product
    for product in page:
        slug = {}
        slug['category'] = category
        slug['href'] = product.find('a')['href']
        slugs.append(slug)
    df = pd.DataFrame(slugs)
    slugs_df = pd.concat([slugs_df, df], axis = 0, ignore_index = True, sort = True)
slugs_df.to_csv('../data/follain_product_slugs.csv', index = False)

In [3]:
slugs_df.head()

,category,href
0,cleansers,/collections/skincare-cleansers/products/brigh...
1,cleansers,/collections/skincare-cleansers/products/osea-...
2,cleansers,/collections/skincare-cleansers/products/tata-...
3,cleansers,/collections/skincare-cleansers/products/one-l...
4,cleansers,/collections/skincare-cleansers/products/ursa-...


In [4]:
#GETTING HTML FILES THAT WILL LEAD TO THE PRODUCT PAGE

for i in slugs_df.index: #for every url slug we got
    time.sleep(20)
    url = "https://follain.com" + slugs_df.loc[i, 'href']
    res = requests.get(url)
    if res.status_code != 200:
        print('error when requestion {}'.format(url))
        break
    soup = BeautifulSoup(res.content, 'html.parser')
    #going to save it to this path
    path = './soups/follainsoup' + str(i) + '.html'
    with open(path, 'w') as f:
        f.write(str(soup))

In [5]:
#GATHER INFO FROM THE PRODUCT HTML PAGES

#opening a new csv file with headers
header = ['name', 'brand', 'category', 'price', 'ingredients']

with open('../data/follain_product_info.csv', "w", newline='') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(header) # write the header
    

for i in slugs_df.index:
    products = [] #empty list to append the dictionary in to before passing in to a DataFrame
    #setting the path to the the html files
    path = './soups/follainsoup' + str(i) + '.html'
    #opening and reading the file
    file_path = open(path, 'rb')
    file_read = file_path.read()
    #creating a new soup from it
    soup = BeautifulSoup(file_read, 'html.parser')
    #gathering the data from the pages
      
              
    product = {}
    product['name'] = soup.find('h1', {'class': 'product-single-name'}).text
    product['brand'] = soup.find('a', {'class': 'product-single-vendor'}).text
    product['category'] = slugs_df.category[i]
    product['price'] = soup.find('div', {'class': 'product-single-price'}).text
    
    list_of_ingredients = []
    for i in soup.find_all('span', {'class': 'ingredient-item'}):
        list_of_ingredients.append(i.text)
    product['ingredients'] = list_of_ingredients
    
    
        #append the empty list to later make in to a dataframe
    products.append(product)
    product_df = pd.DataFrame(products) 
        #append in the csv file that created above
    product_df.to_csv('../data/follain_product_info.csv', mode='a', index = False, header = False)

    
   